# Regressionsanalyse & Modellierung

Nachdem wir in den vorangegangenen Notebooks Zusammenhänge (Korrelationen) und Gruppenunterschiede (Tests) identifiziert haben, wollen wir nun Kausalmodelle bauen. Korrelationen zeigen nur, dass ein Zusammenhang besteht. Die Regression erlaubt uns zu quantifizieren, wie stark sich eine Änderung einer Variablen (z. B. Mordrate) auf unsere Zielvariable (Sicherheitsgefühl) auswirkt.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pathlib import Path
import scipy.stats as stats

In [5]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Daten laden & Vorbereiten

In [6]:
# Daten laden (robuster Pfad)
base_dir = Path.cwd()
data_file = 'oecd_snapshot_latest.csv'
candidates = [
    base_dir / 'data' / data_file,
    base_dir.parent / 'data' / data_file,
]
data_path = next((p for p in candidates if p.exists()), None)
if data_path is None:
    raise FileNotFoundError(f"Datei nicht gefunden. Versucht wurden: {candidates}")
print(f"Lade Daten von: {data_path}")
df = pd.read_csv(data_path)

# Pivotisieren (Wide Format für Regression nötig)
# Wir wählen relevante Variablen basierend auf der Korrelationsanalyse (NB 03)
cols_of_interest = ['Feeling safe at night', 'Homicides', 'Social support', 'Life satisfaction']
df_filtered = df[df['measure'].isin(cols_of_interest)].copy()
df_filtered['value'] = pd.to_numeric(df_filtered['value'], errors='coerce')

# Mehrfachmessungen (z.B. verschiedene Jahre) mitteln, damit das Pivot klappt
df_pivot = df_filtered.pivot_table(
    index='reference_area',
    columns='measure',
    values='value',
    aggfunc='mean'
)
df_pivot = df_pivot.dropna() # Regressionen vertragen keine Missing Values

# Spalten umbenennen (Leerzeichen entfernen für statsmodels Formeln)
df_pivot.columns = ['FeelingSafe', 'Homicides', 'LifeSat', 'SocialSupport']

print(f"Anzahl der Länder im Modell: {len(df_pivot)}")
df_pivot.head()


ValueError: Index contains duplicate entries, cannot reshape

## 2. Einfache Lineare Regression

Aus Notebook 03 und 04 wissen wir, dass die Mordrate (Homicides) ein starker negativer Prädiktor für das Sicherheitsgefühl ist. Wir quantifizieren diesen Effekt nun.

Modellgleichung: $FeelingSafe = \beta_0 + \beta_1 \cdot Homicides + \varepsilon$

In [ ]:
# Modell definieren und fitten
model_simple = ols('FeelingSafe ~ Homicides', data=df_pivot).fit()

# Ergebnisse ausgeben
print(model_simple.summary())

# Visualisierung der Regressionsgeraden
plt.figure(figsize=(10, 6))
sns.regplot(x='Homicides', y='FeelingSafe', data=df_pivot, 
            scatter_kws={'alpha':0.6}, line_kws={'color':'red'})
plt.title('Einfache Lineare Regression: Einfluss der Mordrate auf das Sicherheitsgefühl')
plt.xlabel('Mordrate (pro 100k Einwohner)')
plt.ylabel('Sicherheitsgefühl (%)')
plt.show()

NameError: name 'df_pivot' is not defined

## 3. Modell-Optimierung: Log-Transformation

Unsere Analyse in Notebook 02 hat gezeigt, dass Homicides extrem rechtsschief ist und Ausreißer hat. Lineare Regression setzt zwar keine Normalverteilung der Variablen voraus, aber Ausreißer in X haben einen enormen Leverage auf die Regressionsgerade. Eine Log-Transformation der Mordrate kann diesen Einfluss dämpfen und den Zusammenhang linearisieren.

Modellgleichung: $FeelingSafe = \beta_0 + \beta_1 \cdot \log(Homicides) + \varepsilon$

In [ ]:
# Log-Transformation der Mordrate
df_pivot['Log_Homicides'] = np.log(df_pivot['Homicides'])

# Neues Modell fitten
model_log = ols('FeelingSafe ~ Log_Homicides', data=df_pivot).fit()

print(model_log.summary())

# Visualisierung mit transformierter Variable
plt.figure(figsize=(10, 6))
sns.regplot(x='Log_Homicides', y='FeelingSafe', data=df_pivot, 
            scatter_kws={'alpha':0.6}, line_kws={'color':'green'})
plt.title('Optimiertes Modell: Sicherheitsgefühl vs. Log(Mordrate)')
plt.xlabel('Log(Mordrate)')
plt.ylabel('Sicherheitsgefühl (%)')
plt.show()

In [ ]:
# Multiple Regression: Log(Mordrate) + Soziale Unterstützung
if 'Log_Homicides' not in df_pivot.columns:
    df_pivot['Log_Homicides'] = np.log(df_pivot['Homicides'])

# Sicherstellen, dass keine Inf/NaN in den Prädiktoren sind
df_multi = df_pivot[['FeelingSafe', 'Log_Homicides', 'SocialSupport']].replace([np.inf, -np.inf], np.nan).dropna()

model_multi = ols('FeelingSafe ~ Log_Homicides + SocialSupport', data=df_multi).fit()

print(model_multi.summary())
